In [2]:
from pprint import pprint
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
# import functions
import autosklearn.classification
import autosklearn.metrics
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

# Load Data

load the training and test data in a format specified by task description

In [3]:
with open("./Dataset/x_test.txt") as file:
    X_test = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/x_train.txt") as file:
    X_train = [[float(digit) for digit in line.split()] for line in file]


with open("./Dataset/y_train.txt") as file:
    y_train = [[float(digit) for digit in line.split()] for line in file]

In the X_test there are 500 features and 5000 observations, y_train contains 5000 values, X_train contains 500 features and 5000 observations

In [4]:
print("X_test datapoints:", len(X_test), " features:", len(X_test[0]))
print("X_train datapoints:", len(X_train), " features:", len(X_train[0]))
print("y_train datapoints:", len(y_train))

X_test datapoints: 5000  features: 500
X_train datapoints: 5000  features: 500
y_train datapoints: 5000


In [11]:

def getScoreLimited(y_true, y_pred):
    """Get score based on y_true, y_pred and number of feature used

    Scoring function, based on which, the best model is selected.
    Score is calculated according to task description: +10 points for each
    correctly classified positive class, -200 points for each feature used
    """

    amounToTake = int(0.45 * len(y_true[y_true == 1]))

    df = pd.DataFrame(y_pred[:, 1], columns=["result"])
    sortedDf = df.sort_values(by="result", ascending=False).head(amounToTake)

    y_predicted = np.zeros_like(y_true)

    y_predicted[sortedDf.index] = 1

    score = 0

    correct = 0

    for i, y in enumerate(y_true):
        if y == 1 and y == y_predicted[i]:
            correct += 1

    score = 10 * correct - 200 * 2
    return score / (amounToTake * 10 - 400)



# Auto

In [6]:
y_train_ravel = np.ravel(y_train, order="C")
y_train_ravel = y_train_ravel.astype(int)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import (
    SelectKBest,
    RFE,
    RFECV,
    SelectFpr,
    SelectFdr,
    SelectFwe,
)
from sklearn.preprocessing import PolynomialFeatures, RobustScaler, StandardScaler

selector = SelectKBest(k=2)

X_new = selector.fit_transform(X_train, y_train_ravel)
X_new = RobustScaler().fit_transform(X_new, y_train_ravel)
X_t, X_te, y_t, y_te = train_test_split(
    X_new, y_train_ravel, random_state=42, test_size=0.33
)

accuracy_scorer = autosklearn.metrics.make_scorer(
    name="accu",
    score_func=getScoreLimited,
    optimum=1,
    greater_is_better=True,
    needs_proba=True,
    needs_threshold=False,
)

## V2

In [23]:
automl = AutoSklearn2Classifier(
    time_left_for_this_task=86400,
    delete_tmp_folder_after_terminate=False,
    metric=autosklearn.metrics.roc_auc,
    seed=42
)
pd.DataFrame.iteritems = pd.DataFrame.items
automl.fit(X_t, y_t, dataset_name="project")

/home/patryk/repos/advml-proj-2/.venv/lib/python3.9/site-packages/smac/intensification/parallel_scheduling.py:153: UserWarning: SuccessiveHalving is executed with 1 workers only. Consider to use pynisher to use all available workers.
  warnings.warn(


[WARNING] [2024-05-30 16:55:19,162:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-05-30 16:55:20,229:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-05-30 16:55:21,388:Client-EnsembleBuilder] No runs were available to build an ensemble from


AutoSklearn2Classifier(delete_tmp_folder_after_terminate=False, metric=roc_auc,
                       per_run_time_limit=12, seed=42,
                       time_left_for_this_task=120)

In [27]:
automl.leaderboard()

,rank,ensemble_weight,type,cost,duration,config_id,train_loss,seed,start_time,end_time,budget,status,data_preprocessors,feature_preprocessors,balancing_strategy,config_origin
model_id,,,,,,,,,,,,,,,,
24,1,0.02,gradient_boosting,0.460279,2.385873,23,0.009371,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],none,Initial design
17,2,0.10,extra_trees,0.462705,2.391402,16,0.127356,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],none,Initial design
30,3,0.02,extra_trees,0.462874,3.916786,29,0.005467,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],weighting,Initial design
31,4,0.04,mlp,0.480629,3.556101,30,0.457383,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],none,Initial design
9,5,0.04,mlp,0.481285,2.524077,8,0.466719,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],none,Initial design
26,6,0.02,mlp,0.484006,3.417205,25,0.460569,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],none,Initial design
20,7,0.20,mlp,0.484470,3.669278,19,0.482207,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],weighting,Initial design
7,8,0.42,gradient_boosting,0.488867,1.902128,6,0.439817,0,1.717081e+09,1.717081e+09,6.25,StatusType.DONOTADVANCE,[],[no_preprocessing],none,Initial design
12,9,0.04,mlp,0.495034,4.462073,11,0.479302,0,1.717081e+09,1.717081e+09,6.25,StatusType.SUCCESS,[],[no_preprocessing],none,Initial design


In [28]:
automl.show_models()

{7: {'model_id': 7,
  'rank': 1,
  'cost': 0.48886731355442303,
  'ensemble_weight': 0.42,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f864d3b6dc0>,
  'balancing': Balancing(random_state=42),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7f864d3b0610>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x7f864d38d550>,
  'sklearn_classifier': None},
 9: {'model_id': 9,
  'rank': 2,
  'cost': 0.4812854802923116,
  'ensemble_weight': 0.04,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f864dd18520>,
  'balancing': Balancing(random_state=42),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7f864d3a1100>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x7f864d3a1460>,
  'sklearn

In [24]:
ypred = automl.predict_proba(X_te)

In [25]:
getScoreLimited(y_te, ypred)

0.4740061162079511

In [31]:
automl.cv_results_['params'][np.argmax(automl.cv_results_['mean_test_score'])]

{'balancing:strategy': 'weighting',
 'classifier:__choice__': 'gradient_boosting',
 'data_preprocessor:__choice__': 'feature_type',
 'feature_preprocessor:__choice__': 'no_preprocessing',
 'classifier:gradient_boosting:early_stop': 'off',
 'classifier:gradient_boosting:l2_regularization': 0.3823866149047987,
 'classifier:gradient_boosting:learning_rate': 0.9564154505530591,
 'classifier:gradient_boosting:loss': 'auto',
 'classifier:gradient_boosting:max_bins': 255,
 'classifier:gradient_boosting:max_depth': 'None',
 'classifier:gradient_boosting:max_leaf_nodes': 50,
 'classifier:gradient_boosting:min_samples_leaf': 5,
 'classifier:gradient_boosting:scoring': 'loss',
 'classifier:gradient_boosting:tol': 1e-07,
 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'median',
 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'none'}

## V1

In [32]:
automl1 = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=86400,
    delete_tmp_folder_after_terminate=False,
    metric=accuracy_scorer,
    seed=42
)
pd.DataFrame.iteritems = pd.DataFrame.items
automl1.fit(X_t, y_t, dataset_name="project")

[WARNING] [2024-05-30 17:05:56,738:Client-AutoMLSMBO(42)::project] Could not find meta-data directory /home/patryk/repos/advml-proj-2/.venv/lib/python3.9/site-packages/autosklearn/metalearning/files/accu_binary.classification_dense


In [ ]:
automl1.cv_results_['params'][np.argmax(automl.cv_results_['mean_test_score'])]

In [ ]:
ypred = automl1.predict_proba(X_te)
getScoreLimited(y_te, ypred)

In [ ]:
automl.leaderboard()